In [36]:
import requests
import time
from json import JSONDecodeError
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# API endpoint
url = "http://data4library.kr/api/loanItemSrch?authKey=f1fc1a713d02a17470687b4edd41210f1fc6568183b647d27bed439dcacd7fd1"

In [7]:
# Parameters
params = {
    "startDt": "2018-01-01",
    "endDt": "2018-12-31",
    "gender": '0',
    "age" : '0',
    "kdc100" : '0',
    "format" : 'json'
}

In [28]:
# 연도 : 2018 ~ 2022
# 성별 : 여성 남성 미상
# 나이 : 20대 ~ 60대 이상 및 미상
datas = []
for start_day in ['2018-01-01','2019-01-01','2020-01-01','2021-01-01','2022-01-01']:
    end_day = start_day[0:4] + '-12-31'
    params['startDt'] = start_day
    params['endDt'] = end_day
    for gender in [0,1,2]:
        params['gender'] = str(gender)
        for age in [20,30,40,50,60,-1]:
            params['age'] = str(age)
            response = requests.get(url, params=params)
            time.sleep(5)
            
            if response.status_code == 200:
                try:
                    data = response.json()
                    date = data['response']['request']['startDt']
                    print(date[0:4])
                    print(data['response']['request']['gender'])
                    print(data['response']['request']['age'])
                    print(data['response']['resultNum'])
                    print('--------------------------------------------')
                    datas.append(data)
                except JSONDecodeError:
                    print("Failed to decode json from response.")
                    print(response.text)
            else:
                print("Error:", response.status_code)
# 쥐피티에 나와 있는 내용 보고 오류 수정하기

2018
0
20
200
--------------------------------------------
2018
0
30
200
--------------------------------------------
2018
0
40
200
--------------------------------------------
2018
0
50
200
--------------------------------------------
2018
0
60
200
--------------------------------------------
2018
0
-1
200
--------------------------------------------
2018
1
20
200
--------------------------------------------
2018
1
30
200
--------------------------------------------
2018
1
40
200
--------------------------------------------
2018
1
50
200
--------------------------------------------
2018
1
60
200
--------------------------------------------
2018
1
-1
200
--------------------------------------------
2018
2
20
200
--------------------------------------------
2018
2
30
200
--------------------------------------------
2018
2
40
200
--------------------------------------------
2018
2
50
200
--------------------------------------------
2018
2
60
200
------------------------------------------

In [47]:
len(datas)

90

In [52]:
# 그룹 이름 지정 및 책 목록 리스트 만들기
group_dict = {}
group_gender=''
for data in datas:
    booknames = [doc['doc']['bookname']for doc in data['response']['docs']]
    group_age = data['response']['request']['age']
    group_gender = data['response']['request']['gender']
    date = data['response']['request']['startDt']
    group_year = date[0:4]
    if group_gender == '0' :
        group_gender = 'male'
    elif group_gender == '1' :
        group_gender = 'female'
    else :
        group_gender = 'unknown'
    group_name = group_year + '_' + group_gender + '_' + group_age
    group_dict[group_name] = booknames

In [53]:
df = pd.DataFrame(group_dict)
df.to_excel('list_of_books_by_groups.xlsx', index = False)